In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as sm
from sklearn.linear_model import LinearRegression


In [2]:
df = pd.read_csv("Ofertas_finales.csv",sep="|")

In [3]:
#### Preproccesing
def DroppingVariables(var: list, df):
    return df.drop(var,axis=1)


In [6]:
df = DroppingVariables(['ciudad','Longitud','Latitud',
                'Administración','Descripcion'],df)

In [7]:
%pwd

'd:\\PABLO\\Streamapp'

In [8]:
X = df.drop(['Canon'],axis=1)
y= df['Canon']

In [9]:
col_categoricos = X.select_dtypes("object").columns
col_numericos = X.select_dtypes(np.number).columns

In [10]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor

In [12]:
#['estrato_0',
#'estrato_1','estrato_2','estrato_3','estrato_4','estrato_5','estrato_6','estrato_7','Admin_NO','Admin_Si']

In [13]:
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()
preprocessor = ColumnTransformer(
    transformers=[
        ("categorical",categorical_transformer,col_categoricos),
            ("numerico",numeric_transformer, col_numericos)
        ]
)
clf = Pipeline(
    steps=[('preprocessor',preprocessor)]
)

clf_modelo = Pipeline(
    steps=[("Preprocesando",clf),("Modelando",RandomForestRegressor(random_state=19))]
)

#### 
from sklearn.model_selection import train_test_split
import joblib
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

#### to export my preproccesor
clf.fit(X_train)
joblib.dump(clf, "transformador.pkl")

###
clf_modelo.fit(X_train,y_train)
joblib.dump(clf_modelo,"Modelo.pkl")

['Modelo.pkl']

In [137]:
from sklearn.metrics import mean_squared_error, r2_score

In [141]:
predictions = clf_modelo.predict(X_test)

In [142]:
r2 = r2_score(y_test,predictions)

In [143]:
r2

-0.0001714593117931873

In [164]:
dict(zip(y,predictions))

{1300000.0: 16304117.730439449,
 1250000.0: 9172146.745483562,
 1150000.0: 13784952.319601614,
 1350000.0: 7171117.418331403,
 1800000.0: 13074903.80060151,
 1100000.0: 13784952.319601614,
 1200000.0: 10036313.637344837,
 1050000.0: 10639231.833748251,
 1400000.0: 31658444.23254073,
 2000000.0: 7171117.418331403,
 700000.0: 9172146.745483562,
 750000.0: 10639231.833748251,
 5000000.0: 9172146.745483562,
 10000000.0: 7766953.144029062,
 2380000.0: 7171117.418331403,
 24980000.0: 9172146.745483562,
 12000000.0: 7171117.418331403,
 15000000.0: 10639231.833748251,
 2500000.0: 31658444.23254073,
 6000000.0: 31658444.23254073,
 13000000.0: 7171117.418331403,
 2400000.0: 7171117.418331403,
 20000000.0: 7171117.418331403,
 3500000.0: 10639231.833748251,
 4500000.0: 7171117.418331403,
 3800000.0: 31658444.23254073,
 3400000.0: 7171117.418331403,
 1500000.0: 13784952.319601614,
 330000.0: 13784952.319601614,
 1600000.0: 9172146.745483562,
 950000.0: 7328218.249295253,
 8600000.0: 13784952.319601

In [148]:
import statsmodels.api as sm

In [149]:
tranform = clf.transform(X_train)

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [162]:
X_prueba = X_train.drop(['ID_Estrato','Incluye_admin','Antiguedad','Piso',
       'Habitaciones','Banios','Parqueadero'],axis=1)
X_prueba = sm.add_constant(X_prueba)

In [163]:
modelo = sm.OLS(y_train,X_prueba).fit()
print(modelo.summary())


                            OLS Regression Results                            
Dep. Variable:                  Canon   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.785
Date:                Tue, 10 Jan 2023   Prob (F-statistic):              0.168
Time:                        19:01:06   Log-Likelihood:            -5.4898e+05
No. Observations:               25957   AIC:                         1.098e+06
Df Residuals:                   25954   BIC:                         1.098e+06
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const            9.666e+06   3.06e+06     